In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras import layers as l

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
# Converting Text column 'Sex' to numeric
df_train.Sex = pd.get_dummies(df_train['Sex'], columns='Sex')

In [6]:
# Making sure null values and zeros are erradicated from column 'Age'. Such places will be occupied by the mean for it 
# to be more convenient rather than having null values or zeros for passenger age
df_train.Age = df_train.Age.replace(0, np.NaN)
mean_age = int(df_train['Age'].mean())
df_train.Age = df_train.Age.replace(np.NaN, mean_age)
df_train.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.352413,29.560236,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,0.477990,13.005010,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,0.000000,29.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
# Categorizing and converting the 'Embarked' column to corresponding numeric values
df_train.Embarked = pd.Categorical(df_train.Embarked, ('C', 'Q', 'S'), ordered=True)
df_train.Embarked = df_train.Embarked.cat.codes

In [8]:
# Wiping out unnecessary details from the train dataset
df_train = df_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
df_train.head(63)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,2
1,1,1,1,38.0,1,0,71.2833,0
2,1,3,1,26.0,0,0,7.9250,2
3,1,1,1,35.0,1,0,53.1000,2
4,0,3,0,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
58,1,2,1,5.0,1,2,27.7500,2
59,0,3,0,11.0,5,2,46.9000,2
60,0,3,0,22.0,0,0,7.2292,0
61,1,1,1,38.0,0,0,80.0000,-1


In [9]:
# Making sure column 'Embarked' does not contain NULL values
mean_embarked = int(df_train['Embarked'].mean())
df_train.Embarked = df_train.Embarked.replace(-1, mean_embarked)

In [10]:
df_train['family_size'] = df_train.SibSp + df_train.Parch + 1
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size
0,0,3,0,22.0,1,0,7.2500,2,2
1,1,1,1,38.0,1,0,71.2833,0,2
2,1,3,1,26.0,0,0,7.9250,2,1
3,1,1,1,35.0,1,0,53.1000,2,2
4,0,3,0,35.0,0,0,8.0500,2,1
...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,2,1
887,1,1,1,19.0,0,0,30.0000,2,1
888,0,3,1,29.0,1,2,23.4500,2,4
889,1,1,0,26.0,0,0,30.0000,0,1


In [11]:
max(df_train['family_size'])

11

In [12]:
def fam_strength(fam_size):
    if fam_size == 1:
#         alone
        return 0
    elif fam_size <=4:
#         medium
        return 1
    else:
#         strong
        return 2


In [13]:
df_train['family_strength'] = df_train.family_size.apply(fam_strength)

# temp = df_train['family_size'].apply(fam_strength)
# temp

# for i in df_train['family_size']:
#     if i == 1:
# #         alone
#         strength = 0
#     elif i <=5:
# #         medium
#         strength = 1
#     else:
# #         strong
#         strength = 2
#     df_train['family_strength'].append(strength)

In [14]:
df_train['alone'] = [1 if i == 1 else 0 for i in df_train.family_size]
df_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size,family_strength,alone
0,0,3,0,22.0,1,0,7.2500,2,2,1,0
1,1,1,1,38.0,1,0,71.2833,0,2,1,0
2,1,3,1,26.0,0,0,7.9250,2,1,0,1
3,1,1,1,35.0,1,0,53.1000,2,2,1,0
4,0,3,0,35.0,0,0,8.0500,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,2,1,0,1
887,1,1,1,19.0,0,0,30.0000,2,1,0,1
888,0,3,1,29.0,1,2,23.4500,2,4,1,0
889,1,1,0,26.0,0,0,30.0000,0,1,0,1


In [15]:
y = df_train['Survived']
x = df_train.drop(['Survived'], axis=1)

In [16]:
from sklearn.model_selection import train_test_split

# Splitting data to train and test where 10% will be used to test and the rest to be trained
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [17]:
df_train[['Age', 'Survived']].groupby(['Age'], as_index=False).mean().sort_values(by='Survived')

,Age,Survived
77,61.00,0.0
30,23.50,0.0
37,28.50,0.0
40,30.50,0.0
26,20.50,0.0
...,...,...
3,0.83,1.0
2,0.75,1.0
1,0.67,1.0
68,53.00,1.0


In [18]:
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
1,1,0.742038
0,0,0.188908


Cleaning the test dataset same as the train dataset

In [19]:
df_test.Sex = pd.get_dummies(df_test['Sex'])
df_test.Embarked = pd.Categorical(df_test.Embarked, ('C', 'Q', 'S'), ordered=True)
df_test.Embarked = df_test.Embarked.cat.codes
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,2
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,1
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,2
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,2


In [20]:
passenger_id = df_test['PassengerId']
df_test = df_test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,1
1,3,1,47.0,1,0,7.0000,2
2,2,0,62.0,0,0,9.6875,1
3,3,0,27.0,0,0,8.6625,2
4,3,1,22.0,1,1,12.2875,2


In [21]:
df_test.Age = df_test.Age.replace(0, np.NaN)
test_mean_age = int(df_test.Age.mean())
df_test.Age = df_test.Age.replace(np.NaN, test_mean_age)

In [22]:
test_mean_fare = df_test.Fare.mean()
df_test.Fare = df_test.Fare.replace(np.NaN, test_mean_fare)

In [23]:
test_mean_embarked = int(df_test.Embarked.mean())
df_test.Embarked = df_test.Embarked.replace(-1, test_mean_embarked)
df_test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.363636,30.216507,0.447368,0.392344,35.627188,1.401914
std,0.841838,0.481622,12.635016,0.896760,0.981429,55.840500,0.854496
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,23.000000,0.000000,0.000000,7.895800,1.000000
50%,3.000000,0.000000,30.000000,0.000000,0.000000,14.454200,2.000000
75%,3.000000,1.000000,35.750000,1.000000,0.000000,31.500000,2.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


In [24]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier

In [25]:
# scores_ = []
# highest_score = 0
# estimator = 0
# for i in range (1, 25):
#     rnd_forest = RandomForestClassifier(n_estimators=i)
#     rnd_forest.fit(x, y)
#     scores_.append(round(rnd_forest.score(x_test, y_test), 2))
#     if scores_[i-1] >= highest_score:
#         highest_score = scores_[i-1]
#         estimator = i
# scores_

In [26]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# # Plotting a graph with respect to n_estimators and scores
# plt.plot([i for i in range(1, 25)], scores_, color='blue')

# for i in range(1, 25):
#     plt.text(i, scores_[i - 1], (i, scores_[i - 1]))

In [27]:
# # Training the dataset for prediction
# model = DecisionTreeClassifier()
# model.fit(x, y)
# y_pred = model.predict(df_test)
# print(highest_score)
# print(estimator)

In [28]:
# model.score(df_test, y_pred)

In [29]:
# # Final dataset after prediction
# df_test['Survived'] = y_pred
# df_test.head()

In [30]:
# # Generating the submission file as a csv
# submit_prediction = pd.DataFrame({
#     'PassengerId': passenger_id,
#     'Survived': y_pred
# })

# submit_prediction.to_csv('submission.csv', index=False)

In [31]:
df_train.keys()

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'family_size', 'family_strength', 'alone'],
      dtype='object')

In [32]:
df_test['family_size'] = df_test.SibSp + df_test.Parch + 1
df_test['family_strength'] = df_test.family_size.apply(fam_strength)
df_test['alone'] = [1 if i == 1 else 0 for i in df_test.family_size]
df_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,family_size,family_strength,alone
0,3,0,34.5,0,0,7.8292,1,1,0,1
1,3,1,47.0,1,0,7.0000,2,2,1,0
2,2,0,62.0,0,0,9.6875,1,1,0,1
3,3,0,27.0,0,0,8.6625,2,1,0,1
4,3,1,22.0,1,1,12.2875,2,3,1,0
...,...,...,...,...,...,...,...,...,...,...
413,3,0,30.0,0,0,8.0500,2,1,0,1
414,1,1,39.0,0,0,108.9000,0,1,0,1
415,3,0,38.5,0,0,7.2500,2,1,0,1
416,3,0,30.0,0,0,8.0500,2,1,0,1


In [189]:
model = k.Sequential([
    l.Flatten(),
    l.Dense(24, activation=tf.nn.relu),
#     l.Dense(24, activation=tf.nn.softmax),
#     l.Dense(16, activation=tf.nn.sigmoid),
    l.Dense(2, activation=tf.nn.sigmoid)
])

In [199]:
model.compile(optimizer=tf.optimizers.Adam(),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [200]:
model.fit(np.array(x_train), np.array(y_train), epochs=6, validation_data=(x_test, y_test))

Train on 801 samples, validate on 90 samples
Epoch 1/6
801/801 [==============================] - 0s 530us/sample - loss: 0.4387 - accuracy: 0.8165 - val_loss: 0.4735 - val_accuracy: 0.8222
Epoch 2/6
801/801 [==============================] - 0s 105us/sample - loss: 0.4126 - accuracy: 0.8202 - val_loss: 0.4585 - val_accuracy: 0.8000
Epoch 3/6
801/801 [==============================] - 0s 105us/sample - loss: 0.4168 - accuracy: 0.8177 - val_loss: 0.4766 - val_accuracy: 0.8000
Epoch 4/6
801/801 [==============================] - 0s 106us/sample - loss: 0.4242 - accuracy: 0.8102 - val_loss: 0.4426 - val_accuracy: 0.8111
Epoch 5/6
801/801 [==============================] - 0s 104us/sample - loss: 0.4115 - accuracy: 0.8140 - val_loss: 0.4413 - val_accuracy: 0.7889
Epoch 6/6
801/801 [==============================] - 0s 108us/sample - loss: 0.4121 - accuracy: 0.8140 - val_loss: 0.4620 - val_accuracy: 0.8333


In [201]:
y_pred = model.predict(df_test)
survived = []
for j in range(0, len(y_pred)):
    survived.append(np.argmax(y_pred[j]))


In [202]:
submit_prediction = pd.DataFrame({
    'PassengerId': passenger_id,
    'Survived': survived
})

submit_prediction.to_csv('submission.csv', index=False)